## Glass training
 This notebook loads the Glass identification dataset (https://archive.ics.uci.edu/dataset/42/glass+identification), preprocesses it and trains a simple model

In [1]:
import sys
import os
PROJ_DIR = os.path.realpath(os.path.dirname(os.path.abspath('')))
sys.path.append(os.path.join(PROJ_DIR,'src'))



Load data from CSV and save it to a suitable format. This can be skipped if concrete_data.npz is in assets.

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def load_glass(path):
    df = pd.read_csv(path, delimiter=',', header=None)

    labels = df[10]
    labels = df[10]
    x = df.drop(columns=10)
    x = x.drop(columns=0)

    possible_labels = labels.unique().tolist()
    print(possible_labels)
    print(len(possible_labels))
    y = labels.map(lambda x: possible_labels.index(x))

    return x.to_numpy(), y.to_numpy()
    
x, y = load_glass(os.path.join(PROJ_DIR, 'assets', 'data', 'glass.data'))

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

FileNotFoundError: [Errno 2] No such file or directory: '/home/lidia/eval-project/assets/data/glass.data'

In [10]:
# Save to assets
np.savez(os.path.join(PROJ_DIR, 'assets', 'data', 'glass'),\
        x_train=x_train,\
        x_test=x_test,\
        y_train=y_train,\
        y_test=y_test)

Load data from file

In [4]:
file_data = np.load(os.path.join(PROJ_DIR, 'assets', 'data', 'glass.npz'))
x_train = file_data['x_train']
x_test = file_data['x_test']
y_train = file_data['y_train']
y_test = file_data['y_test']

In [24]:
import torch
class RandomMasker(torch.nn.Module):
    def __init__(self, masking_value:torch.tensor):
        super().__init__()
        self._masking_value = masking_value
    def forward(self, x): # Assumes inputs are (batch_size, num_vars)
        selection_levels = torch.rand((x.shape[0], 1)) # A different selection level for each element of the batch
        selected_pixels = torch.le(torch.rand(x.shape), selection_levels) # A different selection level for each element of the batch
        return x * selected_pixels + self._masking_value * ~selected_pixels

# Zeros
masking_value = np.zeros(x_train.shape[1])
# Mean
masking_value = np.mean(x_train, axis=0)
masker = RandomMasker(torch.tensor(masking_value).float())

Train a MLP model

In [25]:
MODEL_NEURONS = 100
MODEL_EPOCHS= 20000
MODEL_LR = 5e-3
MODEL_LABEL_NUM = len(np.unique(y_train))

class MLP(torch.nn.Module):
    def __init__(self, n_neurons):
        super(MLP, self).__init__()
        self.fc1 = torch.nn.Linear(x_train.shape[1], n_neurons)
        self.ac1 = torch.nn.Sigmoid()
        self.fc2 = torch.nn.Linear(n_neurons, MODEL_LABEL_NUM)
        self.ac2 = torch.nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.ac1(x)
        logits = self.fc2(x)
        x = self.ac2(logits)
        return x

x_train_tensor = torch.tensor(x_train).float()
y_train_tensor = torch.tensor(y_train)
x_test_tensor = torch.tensor(x_test).float()
y_test_tensor = torch.tensor(y_test)

label_onehot = torch.zeros(y_train.shape[0], MODEL_LABEL_NUM)
label_onehot.scatter_(1, y_train_tensor.unsqueeze(1), 1)
class_weights = 1.0/label_onehot.mean(axis=0)

network = MLP(MODEL_NEURONS)
loss = torch.nn.BCELoss(weight=class_weights)
optimizer = torch.optim.Adam(network.parameters(), lr=MODEL_LR, weight_decay=1e-3)

#MODEL_EPOCHS = 0 # Untrained
for epoch in range(MODEL_EPOCHS):
    x_train_tensor_masked = masker(torch.tensor(x_train).float()) # A different set of RandomMasks for each batch
    optimizer.zero_grad()
    
    preds = network(x_train_tensor_masked)
    label_onehot = torch.zeros(y_train.shape[0], MODEL_LABEL_NUM)
    label_onehot.scatter_(1, y_train_tensor.unsqueeze(1), 1)
    loss_value = loss(preds, label_onehot)
    loss_value.backward()        
    optimizer.step()

    train_accuracy = (preds.argmax(dim=1) == y_train_tensor).float().mean() 

    test_preds = network.forward(x_test_tensor)        
    test_accuracy = (test_preds.argmax(dim=1) == y_test_tensor).float().mean() 
    print(f'Epoch {epoch}/{MODEL_EPOCHS} - Loss: {loss_value.item()} - Train accuracy: {train_accuracy} - Test accuracy: {test_accuracy}')  
    #if test_accuracy > 0.6: # Undertrained
    #    break

test_preds = network.forward(x_test_tensor)        
test_accuracy = (test_preds.argmax(dim=1) == y_test_tensor).float().mean() 
print(test_accuracy.item())


Epoch 0/20000 - Loss: 3.3958663940429688 - Train accuracy: 0.38596490025520325 - Test accuracy: 0.23255814611911774
Epoch 1/20000 - Loss: 3.152305841445923 - Train accuracy: 0.38596490025520325 - Test accuracy: 0.23255814611911774
Epoch 2/20000 - Loss: 3.0478785037994385 - Train accuracy: 0.38596490025520325 - Test accuracy: 0.23255814611911774
Epoch 3/20000 - Loss: 3.0307769775390625 - Train accuracy: 0.38596490025520325 - Test accuracy: 0.23255814611911774
Epoch 4/20000 - Loss: 3.046642303466797 - Train accuracy: 0.38596490025520325 - Test accuracy: 0.23255814611911774
Epoch 5/20000 - Loss: 3.0614609718322754 - Train accuracy: 0.38596490025520325 - Test accuracy: 0.23255814611911774
Epoch 6/20000 - Loss: 3.0686299800872803 - Train accuracy: 0.38596490025520325 - Test accuracy: 0.23255814611911774
Epoch 7/20000 - Loss: 3.0660207271575928 - Train accuracy: 0.38596490025520325 - Test accuracy: 0.23255814611911774
Epoch 8/20000 - Loss: 3.054558277130127 - Train accuracy: 0.38596490025520

In [26]:
# Save model
torch.save(network.state_dict(), os.path.join(PROJ_DIR,'assets','models','glass-ood-mean-mlp.pth'))